In [1]:
# python
import os
import math
import csv

# random
import random
#data analysis libraries
import numpy as np
import pandas as pd

# machine learning
import sklearn

# deep learning
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split
import torch.nn.functional as F
import pytorch_lightning as pl

#d2l
import d2l.torch as d2l

# For plotting learning curve
from torch.utils.tensorboard import SummaryWriter


#visualization libraries
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# For Progress Bar
from tqdm import tqdm

#ignore warnings
import warnings

warnings.filterwarnings('ignore')

# auto load change
%load_ext autoreload
%autoreload 2

In [ ]:
from network.subnet import EntityMask

model = EntityMask(5, 2)

## Improvement
+ use parameter instead of hard-code for **base class and task** settings
+ metaR `batchnorm1d` is canceled by me

## Dataset set up
+ NELL one: 30 base classes, 21 novel classes(3 way 5 shot 7 sessions)

## Dataset and DataLoader
+ 基于 base class or novel class 数量构建 train_task.
use `next_batch` to get train_task shape: `(4, batch_size, num)`
+ every epoch will get same relations(`self.curr_rel_idx` will ensure this)
### construct support and query
random choose support(相同数量 positive and negative) query(相同数量 positive and negative) according to current relation
+ support query shape: `(batch_size, num, 2(e1_e2), embedding_shape)`

## MetaR train

### embedding entity
get embeddings according to entity id 

### relation learner
![](imgs/img.png)
![](imgs/img_4.png)

### gradient descent in support 
![](imgs/img_1.png)
![](imgs/img_2.png)
![](imgs/img_3.png)

## eval model
+ loop all triples in eval 
+ get one query and support per time
+ predict rank

+ If relation is same?

+ self.curr_tri_idx filter by relation(cache past relations)

In [2]:
# TODO: simplify it into a function
import json
import numpy as np
tail = ''
data_dir = {
    'train_tasks_in_train': './NELL/train_tasks_in_train.json',
    'train_tasks': './NELL/continual_train_tasks.json',
    'test_tasks': './NELL/test_tasks.json',
    'dev_tasks': './NELL/continual_dev_tasks.json',

    'rel2candidates_in_train': './NELL/rel2candidates_in_train.json',
    'rel2candidates': './NELL/rel2candidates.json',

    'e1rel_e2_in_train': './NELL/e1rel_e2_in_train.json',
    'e1rel_e2': './NELL/e1rel_e2.json',

    'ent2ids': './NELL/ent2ids',
    'ent2vec': './NELL/ent2vec.npy',
}
dataset = dict()
print("loading train_tasks{} ... ...".format(tail))
dataset['train_tasks'] = json.load(open(data_dir['train_tasks' + tail]))
print("loading test_tasks ... ...")
dataset['test_tasks'] = json.load(open(data_dir['test_tasks']))
print("loading dev_tasks ... ...")
dataset['dev_tasks'] = json.load(open(data_dir['dev_tasks']))
print("loading rel2candidates{} ... ...".format(tail))
dataset['rel2candidates'] = json.load(open(data_dir['rel2candidates' + tail]))
print("loading e1rel_e2{} ... ...".format(tail))
dataset['e1rel_e2'] = json.load(open(data_dir['e1rel_e2' + tail]))
print("loading ent2id ... ...")
dataset['ent2id'] = json.load(open(data_dir['ent2ids']))
dataset['ent2emb'] = np.load(data_dir['ent2vec'])
from data_loader import DataLoader
params = {'few': 1, 'num_query': 3, 'batch_size': 16, 'base_classes_few': 10, 'base_classes_num_query': 30, 'base_classes_relation': 30}
train_data_loader = DataLoader(dataset, params, step='train')
dev_data_loader = DataLoader(dataset, params, step='dev')
test_data_loader = DataLoader(dataset, params, step='test')
data_loaders = [train_data_loader, dev_data_loader, test_data_loader]

loading train_tasks ... ...
loading test_tasks ... ...
loading dev_tasks ... ...
loading rel2candidates ... ...
loading e1rel_e2 ... ...
loading ent2id ... ...


In [3]:
train_task, curr_rel = train_data_loader.next_batch(False, True)

In [16]:
mask = (torch.ones(30) > 0).nonzero(as_tuple=True)[0]
mask

tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29])

In [23]:
for i in mask:
    print(i.item())

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29


In [20]:
train_task[0][0]

[['concept:academicfield:science',
  'concept:academicfieldsuchasacademicfield',
  'concept:academicfield:industrial_engineering'],
 ['concept:academicfield:technology',
  'concept:academicfieldsuchasacademicfield',
  'concept:academicfield:engineer'],
 ['concept:academicfield:science',
  'concept:academicfieldsuchasacademicfield',
  'concept:academicfield:materials_science'],
 ['concept:academicfield:technology',
  'concept:academicfieldsuchasacademicfield',
  'concept:academicfield:mathematics'],
 ['concept:academicfield:technology',
  'concept:academicfieldsuchasacademicfield',
  'concept:academicfield:geophysics'],
 ['concept:academicfield:science',
  'concept:academicfieldsuchasacademicfield',
  'concept:academicfield:international_studies'],
 ['concept:academicfield:technology',
  'concept:academicfieldsuchasacademicfield',
  'concept:academicfield:biology'],
 ['concept:academicfield:technology',
  'concept:academicfieldsuchasacademicfield',
  'concept:academicfield:industrial_en

In [8]:
eval_triples = []
all_rels = sorted(list(dataset['dev_tasks'].keys()))
for rel in all_rels:
    eval_triples.extend(dataset['dev_tasks'][rel][3:])

In [24]:
# eval_triples = []
eval_triples.extend(dataset['dev_tasks'][all_rels[0]][:3])


In [25]:
eval_triples

[['concept:academicfield:management_program',
  'concept:academicfieldsuchasacademicfield',
  'concept:academicfield:business'],
 ['concept:academicfield:social_work',
  'concept:academicfieldsuchasacademicfield',
  'concept:academicfield:business'],
 ['concept:academicfield:biology',
  'concept:academicfieldsuchasacademicfield',
  'concept:politicsissue:environment'],
 ['concept:academicfield:management_program',
  'concept:academicfieldsuchasacademicfield',
  'concept:academicfield:business'],
 ['concept:academicfield:social_work',
  'concept:academicfieldsuchasacademicfield',
  'concept:academicfield:business'],
 ['concept:academicfield:biology',
  'concept:academicfieldsuchasacademicfield',
  'concept:politicsissue:environment']]

In [19]:
len(dataset['dev_tasks'][all_rels[0]])

92

In [ ]:
# Test relation in eval
# repeat relation is same